In [1]:
# coding: utf-8
import gym
import torch.utils.data as data
from dynamics import *
from controller import *
from utils import *
from quanser_robots.common import GentlyTerminating

# datasets:  numpy array, size:[sample number, input dimension]
# labels:  numpy array, size:[sample number, output dimension]

config_path = "config.yml"
print_config(config_path)
config = load_config(config_path)

env_id ="Qube-v0" # "CartPole-v0"
env = GentlyTerminating(gym.make(env_id))


************************
*** model configuration ***
load_model: false
model_path: storage/exp_1.ckpt
n_actions: 1
n_hidden: 1
n_states: 6
size_hidden: 256
use_cuda: true

*** train configuration ***
batch_size: 70
exp_number: 1
learning_rate: 0.001
n_epochs: 50
save_loss_fig: true
save_loss_fig_frequency: 30
save_model_flag: true
save_model_path: storage/exp_1.ckpt

************************


In [2]:
dynamic_model = DynamicModel(config)

dataset_factory = DatasetFactory(env,config)
dataset_factory.collect_random_dataset()

train_datasets_norm, train_labels_norm = dynamic_model.normlize_datasets(dataset_factory.random_dataset["data"],
                                                                         dataset_factory.random_dataset["label"])


Collect random dataset shape:  (951, 7)


/home/xinye/projects/rl/RL-project/MPC/MPC qube/dynamics.py:187: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if datasets == None:
/home/xinye/projects/rl/RL-project/MPC/MPC qube/dynamics.py:191: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if labels == None:


In [3]:
dynamic_model.train(train_datasets_norm,train_labels_norm)

Total training step per epoch [14]


/home/xinye/anaconda2/envs/pytorch/lib/python3.6/site-packages/torch/nn/_reduction.py:16: UserWarning: reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.
  warnings.warn("reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.")


Epoch [30/50], Loss: 0.45359700


[0.95957403523581364,
 0.69611769914627075,
 0.59186799611364094,
 0.55097734076636173,
 0.536141791513988,
 0.52623537395681652,
 0.52957870491913384,
 0.52431513369083405,
 0.52521657305104397,
 0.52337946849209926,
 0.51842501546655384,
 0.51581865974834984,
 0.52285975004945484,
 0.5177939129727227,
 0.52221991334642681,
 0.51507708430290222,
 0.51289452612400055,
 0.50840265410287044,
 0.51096228616578243,
 0.49901549730982098,
 0.50068067014217377,
 0.49713297188282013,
 0.48889744920389994,
 0.49137013086250853,
 0.48415613813059671,
 0.47079813054629732,
 0.47057926867689404,
 0.46660811560494558,
 0.46062886289187838,
 0.45359699640955242,
 0.44748150876590181,
 0.43519412832600729,
 0.4334685355424881,
 0.42673175462654661,
 0.41299640067986082,
 0.40716907382011414,
 0.39726206873144421,
 0.39266331919601988,
 0.37559998780488968,
 0.37108723819255829,
 0.36052938444273813,
 0.35423309888158527,
 0.34352820579494747,
 0.33201276510953903,
 0.3238756667290415,
 0.314719556697

In [15]:
x = np.arange(7)
dynamic_model.predict(x)

/home/xinye/projects/rl/RL-project/MPC/MPC qube/dynamics.py:55: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.Variable = lambda *args, **kwargs: autograd.Variable(*args, **kwargs).cuda()


array([[-0.046298,  0.078314, -0.056775, -0.110759,  3.766605, -9.145421]], dtype=float32)

In [12]:
y = dynamic_model.predict(x[0])

/home/xinye/projects/rl/RL-project/MPC/MPC qube/dynamics.py:55: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.Variable = lambda *args, **kwargs: autograd.Variable(*args, **kwargs).cuda()


In [17]:
y

(1, 6)

In [ ]:
datasets = {"data":state, "label":action}

In [ ]:
datasets["data"]

In [ ]:
xx = [[1,2,3],[4,5,6],[7,8,9],[10,11,12]]
y= [[1],[4],[7],[10]]
xx = np.array(xx)
y = np.array(y)

In [ ]:
data = np.concatenate((xx,xx),axis = 1)
data

In [ ]:
np.random.shuffle(data)
data